# Baseline model 

Ideas based on analysis:
- Using an average of one of the categorical variables as the sole predictor
- Using a combination of 2 or more (possibly all) of the categorical variables and calculating the average as the predictors
- Be able to incorporate the continuous variables
    + we can calculate the average of each of the values, subtract those averages from the mean of the overall salary data and add it to the prediction

*fitting the numeric data*
1. calculate the mean of the target on the data used for fitting
2. calculate the average salary per each value of the numeric variable (0-25 for years, 0 - 99 for miles)
3. get a difference between the grouped averages and the overall average
4. store it

*predictions with numeric predictors*
(this is after the category predictions are made)
1. after ensuring that the variables are in the data
2. loop over the self.numeric_vars list and join with on = the looping var (key of the dict) 
3. take the rowsum of the category preds and the numeric diff columns 
4. drop category preds and the numeric diff columns 
5. profit

___

In [1]:
import pandas as pd
import numpy as np
from itertools import combinations
import seaborn as sns

from src.eda_utils import salary_per_category_table
from src.Baseline import BaselineModel

# Load data

In [2]:
train_salaries = pd.read_csv("../data/interim/salaries_train_85_15_split.csv", index_col = 0)
test_salaries = pd.read_csv("../data/interim/salaries_test_85_15_split.csv", index_col = 0)

In [3]:
print(f"training set shape: {train_salaries.shape}")
print(f"test set shape: {test_salaries.shape}")

training set shape: (850000, 8)
test set shape: (150000, 8)


# `BaselineModel` Class

*Params*
- constructor:
    + grouping_vars: list of categorical variables to group on for fitting
    + numeric_vars:  list of numerical variables to fit 
    
*Methods*
- fit():
    + For the grouping variables, this calculates the average salary per group and stores this values with each grouping variable as an index
    + For the numeric variables, this groups by each value and calculates the mean and stores it with each value of the variable as the index and the average as the value

In [4]:
# Test prototyping
grouping_vars = ['industry', 'major', 'jobType']

avg_values = salary_per_category_table(grouping_vars, train_salaries)

avg_values.set_index(grouping_vars, inplace=True)

print(avg_values)

# This is the predict stage
train_salaries.join(avg_values, on = grouping_vars, rsuffix = '_preds').head()

                                   salary
industry  major       jobType            
EDUCATION NONE        JANITOR   55.152589
SERVICE   NONE        JANITOR   60.129620
AUTO      NONE        JANITOR   64.680927
EDUCATION NONE        JUNIOR    69.534633
HEALTH    NONE        JANITOR   70.294427
...                                   ...
OIL       ENGINEERING CFO      177.157631
FINANCE   ENGINEERING CEO      177.809339
OIL       BUSINESS    CEO      178.989669
FINANCE   BUSINESS    CEO      185.525577
OIL       ENGINEERING CEO      187.346798

[448 rows x 1 columns]


,jobId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,salary_preds
891719,JOB1362685299406,SENIOR,BACHELORS,PHYSICS,WEB,21,68,128,126.375962
177062,JOB1362684584749,CTO,NONE,NONE,HEALTH,1,14,116,125.447948
27080,JOB1362684434767,SENIOR,HIGH_SCHOOL,NONE,OIL,11,49,133,108.911572
546274,JOB1362684953961,CEO,NONE,NONE,AUTO,17,65,126,129.640336
832069,JOB1362685239756,JUNIOR,HIGH_SCHOOL,NONE,OIL,24,26,137,99.563506


In [31]:
test = BaselineModel(category_vars='jobType')

In [32]:
test.fit(train_salaries)

In [33]:
test.fitted_category_salaries

,salary_preds
jobType,
JANITOR,70.803050
JUNIOR,95.293570
SENIOR,105.453358
MANAGER,115.373455
VICE_PRESIDENT,125.408850
CFO,135.419149
CTO,135.436604
CEO,145.294699


In [34]:
test.predict(test_salaries)

,jobId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,salary_preds
987231,JOB1362685394918,CFO,HIGH_SCHOOL,NONE,WEB,12,21,174,135.419149
79954,JOB1362684487641,JANITOR,HIGH_SCHOOL,NONE,HEALTH,15,64,58,70.803050
567130,JOB1362684974817,CTO,HIGH_SCHOOL,NONE,OIL,21,4,168,135.436604
500891,JOB1362684908578,CTO,HIGH_SCHOOL,NONE,FINANCE,5,89,85,135.436604
55399,JOB1362684463086,JUNIOR,DOCTORAL,BIOLOGY,WEB,23,64,145,95.293570
...,...,...,...,...,...,...,...,...,...
484822,JOB1362684892509,SENIOR,MASTERS,ENGINEERING,HEALTH,21,12,189,105.453358
902986,JOB1362685310673,JANITOR,HIGH_SCHOOL,NONE,OIL,4,28,100,70.803050
138960,JOB1362684546647,VICE_PRESIDENT,NONE,NONE,SERVICE,6,11,87,125.408850
895087,JOB1362685302774,SENIOR,DOCTORAL,CHEMISTRY,EDUCATION,10,13,109,105.453358


In [34]:
avg_salary_overall = train_salaries.salary.mean()

avg_salary_by_year = train_salaries.groupby('yearsExperience').salary.mean()
print(avg_salary_by_year)

# These are the fitted values
print("\n\nThese are the fitted values")
years_diff = avg_salary_by_year - avg_salary_overall
years_diff

yearsExperience
0      91.968565
1      94.039934
2      96.141088
3      97.979226
4     100.037532
5     101.686603
6     103.865457
7     106.107381
8     107.839315
9     110.075783
10    112.134154
11    113.919876
12    116.347211
13    117.870305
14    120.354740
15    121.888924
16    124.041466
17    126.013482
18    128.244769
19    130.103428
20    132.110810
21    134.127544
22    136.309815
23    138.460865
24    140.006013
Name: salary, dtype: float64


These are the fitted values


yearsExperience
0    -24.075262
1    -22.003893
2    -19.902739
3    -18.064601
4    -16.006295
5    -14.357224
6    -12.178370
7     -9.936446
8     -8.204512
9     -5.968044
10    -3.909673
11    -2.123951
12     0.303384
13     1.826478
14     4.310913
15     5.845097
16     7.997639
17     9.969655
18    12.200942
19    14.059601
20    16.066983
21    18.083717
22    20.265988
23    22.417038
24    23.962186
Name: salary, dtype: float64

In [27]:
avg_salary_by_miles = train_salaries.groupby('milesFromMetropolis').salary.mean()

print(avg_salary_by_miles)
miles_diff = avg_salary_by_miles - avg_salary_overall
miles_diff

milesFromMetropolis
0     135.537307
1     135.483163
2     134.640791
3     134.824962
4     135.117070
         ...    
95     98.076253
96     97.344226
97     97.329820
98     96.816241
99     95.816550
Name: salary, Length: 100, dtype: float64


milesFromMetropolis
0     19.493479
1     19.439336
2     18.596964
3     18.781135
4     19.073243
        ...    
95   -17.967574
96   -18.699601
97   -18.714007
98   -19.227586
99   -20.227277
Name: salary, Length: 100, dtype: float64

In [26]:
miles_diff.loc[68]

-7.364727832926263

In [35]:
train_salaries.join(miles_diff, on = 'milesFromMetropolis', rsuffix = '_miles_diff').join(years_diff, on = 'yearsExperience', rsuffix = "_years_diff")

,jobId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,salary_miles_diff,salary_years_diff
891719,JOB1362685299406,SENIOR,BACHELORS,PHYSICS,WEB,21,68,128,-7.364728,18.083717
177062,JOB1362684584749,CTO,NONE,NONE,HEALTH,1,14,116,14.061349,-22.003893
27080,JOB1362684434767,SENIOR,HIGH_SCHOOL,NONE,OIL,11,49,133,0.081878,-2.123951
546274,JOB1362684953961,CEO,NONE,NONE,AUTO,17,65,126,-6.588774,9.969655
832069,JOB1362685239756,JUNIOR,HIGH_SCHOOL,NONE,OIL,24,26,137,9.745740,23.962186
...,...,...,...,...,...,...,...,...,...,...
259178,JOB1362684666865,CTO,HIGH_SCHOOL,NONE,EDUCATION,1,34,113,6.414863,-22.003893
365838,JOB1362684773525,VICE_PRESIDENT,BACHELORS,COMPSCI,EDUCATION,2,38,109,4.619681,-19.902739
131932,JOB1362684539619,SENIOR,BACHELORS,ENGINEERING,EDUCATION,8,19,123,12.417469,-8.204512
671155,JOB1362685078842,CEO,MASTERS,COMPSCI,WEB,9,30,165,8.469702,-5.968044


In [40]:
train_salaries.groupby('yearsExperience')['salary'].mean()

yearsExperience
0      91.968565
1      94.039934
2      96.141088
3      97.979226
4     100.037532
5     101.686603
6     103.865457
7     106.107381
8     107.839315
9     110.075783
10    112.134154
11    113.919876
12    116.347211
13    117.870305
14    120.354740
15    121.888924
16    124.041466
17    126.013482
18    128.244769
19    130.103428
20    132.110810
21    134.127544
22    136.309815
23    138.460865
24    140.006013
Name: salary, dtype: float64

## Run through numeric fit

In [3]:
avg_salary_overall = train_salaries.salary.mean()
avg_salary_overall

116.04382705882352

In [4]:
numeric_vars = ['yearsExperience', 'milesFromMetropolis']

fitted_numeric = {column:None for column in numeric_vars}
fitted_numeric

{'yearsExperience': None, 'milesFromMetropolis': None}

In [7]:
# loop over keys, get the grouped mean, subtract the overall mean from it, store it
for key in fitted_numeric.keys():
    avg = (train_salaries.groupby(key)['salary'].mean()) - avg_salary_overall
    fitted_numeric[key] = avg
    
print(fitted_numeric['yearsExperience'])
print(fitted_numeric['milesFromMetropolis'])

yearsExperience
0    -24.075262
1    -22.003893
2    -19.902739
3    -18.064601
4    -16.006295
5    -14.357224
6    -12.178370
7     -9.936446
8     -8.204512
9     -5.968044
10    -3.909673
11    -2.123951
12     0.303384
13     1.826478
14     4.310913
15     5.845097
16     7.997639
17     9.969655
18    12.200942
19    14.059601
20    16.066983
21    18.083717
22    20.265988
23    22.417038
24    23.962186
Name: salary, dtype: float64
milesFromMetropolis
0     19.493479
1     19.439336
2     18.596964
3     18.781135
4     19.073243
        ...    
95   -17.967574
96   -18.699601
97   -18.714007
98   -19.227586
99   -20.227277
Name: salary, Length: 100, dtype: float64


In [8]:
fitted_numeric

{'yearsExperience': yearsExperience
 0    -24.075262
 1    -22.003893
 2    -19.902739
 3    -18.064601
 4    -16.006295
 5    -14.357224
 6    -12.178370
 7     -9.936446
 8     -8.204512
 9     -5.968044
 10    -3.909673
 11    -2.123951
 12     0.303384
 13     1.826478
 14     4.310913
 15     5.845097
 16     7.997639
 17     9.969655
 18    12.200942
 19    14.059601
 20    16.066983
 21    18.083717
 22    20.265988
 23    22.417038
 24    23.962186
 Name: salary, dtype: float64,
 'milesFromMetropolis': milesFromMetropolis
 0     19.493479
 1     19.439336
 2     18.596964
 3     18.781135
 4     19.073243
         ...    
 95   -17.967574
 96   -18.699601
 97   -18.714007
 98   -19.227586
 99   -20.227277
 Name: salary, Length: 100, dtype: float64}

## Run through numeric predict

In [10]:
# loop through keys, join on the key name
preds = test_salaries.copy()
for key in fitted_numeric.keys():
    column_suffix = f"_{key}_diff"
    preds = preds.join(fitted_numeric[key], on = key, rsuffix = column_suffix)
    
preds

,jobId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,salary_yearsExperience_diff,salary_milesFromMetropolis_diff
987231,JOB1362685394918,CFO,HIGH_SCHOOL,NONE,WEB,12,21,174,0.303384,10.699932
79954,JOB1362684487641,JANITOR,HIGH_SCHOOL,NONE,HEALTH,15,64,58,5.845097,-5.445622
567130,JOB1362684974817,CTO,HIGH_SCHOOL,NONE,OIL,21,4,168,18.083717,19.073243
500891,JOB1362684908578,CTO,HIGH_SCHOOL,NONE,FINANCE,5,89,85,-14.357224,-15.314648
55399,JOB1362684463086,JUNIOR,DOCTORAL,BIOLOGY,WEB,23,64,145,22.417038,-5.445622
...,...,...,...,...,...,...,...,...,...,...
484822,JOB1362684892509,SENIOR,MASTERS,ENGINEERING,HEALTH,21,12,189,18.083717,15.124927
902986,JOB1362685310673,JANITOR,HIGH_SCHOOL,NONE,OIL,4,28,100,-16.006295,8.683477
138960,JOB1362684546647,VICE_PRESIDENT,NONE,NONE,SERVICE,6,11,87,-12.178370,15.127079
895087,JOB1362685302774,SENIOR,DOCTORAL,CHEMISTRY,EDUCATION,10,13,109,-3.909673,14.453634


In [23]:
BL = BaselineModel(category_vars = ['jobType'], numeric_vars = ['yearsExperience', 'milesFromMetropolis'])

BL.fit(train_salaries)

In [24]:
BL.fitted_category_salaries

,salary_preds
jobType,
JANITOR,70.803050
JUNIOR,95.293570
SENIOR,105.453358
MANAGER,115.373455
VICE_PRESIDENT,125.408850
CFO,135.419149
CTO,135.436604
CEO,145.294699


In [5]:
BL.fitted_numeric_salaries

{'yearsExperience': yearsExperience
 0    -24.075262
 1    -22.003893
 2    -19.902739
 3    -18.064601
 4    -16.006295
 5    -14.357224
 6    -12.178370
 7     -9.936446
 8     -8.204512
 9     -5.968044
 10    -3.909673
 11    -2.123951
 12     0.303384
 13     1.826478
 14     4.310913
 15     5.845097
 16     7.997639
 17     9.969655
 18    12.200942
 19    14.059601
 20    16.066983
 21    18.083717
 22    20.265988
 23    22.417038
 24    23.962186
 Name: yearsExperience_diff, dtype: float64,
 'milesFromMetropolis': milesFromMetropolis
 0     19.493479
 1     19.439336
 2     18.596964
 3     18.781135
 4     19.073243
         ...    
 95   -17.967574
 96   -18.699601
 97   -18.714007
 98   -19.227586
 99   -20.227277
 Name: milesFromMetropolis_diff, Length: 100, dtype: float64}

In [8]:
preds = BL.predict(train_salaries, return_only_preds = False, return_all_cols=True)
preds

,jobId,jobType,degree,major,industry,yearsExperience,milesFromMetropolis,salary,salary_preds,yearsExperience_diff,milesFromMetropolis_diff,sum_numeric_diff,mean_numeric_diff,preds_with_sum,preds_with_mean,category_preds_tmp
891719,JOB1362685299406,SENIOR,BACHELORS,PHYSICS,WEB,21,68,128,121.903485,18.083717,-7.364728,10.718989,5.359495,121.903485,116.543991,111.184496
177062,JOB1362684584749,CTO,NONE,NONE,HEALTH,1,14,116,127.799654,-22.003893,14.061349,-7.942544,-3.971272,127.799654,131.770926,135.742198
27080,JOB1362684434767,SENIOR,HIGH_SCHOOL,NONE,OIL,11,49,133,117.750164,-2.123951,0.081878,-2.042073,-1.021036,117.750164,118.771200,119.792236
546274,JOB1362684953961,CEO,NONE,NONE,AUTO,17,65,126,142.539266,9.969655,-6.588774,3.380881,1.690440,142.539266,140.848826,139.158385
832069,JOB1362685239756,JUNIOR,HIGH_SCHOOL,NONE,OIL,24,26,137,143.848817,23.962186,9.745740,33.707926,16.853963,143.848817,126.994854,110.140891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,JOB1362684666865,CTO,HIGH_SCHOOL,NONE,EDUCATION,1,34,113,103.083661,-22.003893,6.414863,-15.589030,-7.794515,103.083661,110.878176,118.672691
365838,JOB1362684773525,VICE_PRESIDENT,BACHELORS,COMPSCI,EDUCATION,2,38,109,93.300437,-19.902739,4.619681,-15.283058,-7.641529,93.300437,100.941966,108.583495
131932,JOB1362684539619,SENIOR,BACHELORS,ENGINEERING,EDUCATION,8,19,123,93.103601,-8.204512,12.417469,4.212957,2.106478,93.103601,90.997123,88.890644
671155,JOB1362685078842,CEO,MASTERS,COMPSCI,WEB,9,30,165,153.184496,-5.968044,8.469702,2.501658,1.250829,153.184496,151.933666,150.682837


In [8]:
from sklearn.metrics import mean_squared_error
print(f"preds_with_sum MSE: {mean_squared_error(preds.salary, preds.preds_with_sum)}")
print(f"preds_with_mean MSE: {mean_squared_error(preds.salary, preds.preds_with_mean)}")

preds_with_sum MSE: 490.4179668796452
preds_with_mean MSE: 575.5115272763995


In [8]:
diff_cols = preds.loc[:,['salary', 'salary_preds','yearsExperience_diff']]
diff_cols

,salary,salary_preds,yearsExperience_diff
891719,128,111.184496,18.083717
177062,116,135.742198,-22.003893
27080,133,119.792236,-2.123951
546274,126,139.158385,9.969655
832069,137,110.140891,23.962186
...,...,...,...
259178,113,118.672691,-22.003893
365838,109,108.583495,-19.902739
131932,123,88.890644,-8.204512
671155,165,150.682837,-5.968044


In [9]:
diff_cols['salary_preds'] + diff_cols['yearsExperience_diff']

891719    129.268213
177062    113.738305
27080     117.668285
546274    149.128040
832069    134.103077
             ...    
259178     96.668799
365838     88.680756
131932     80.686132
671155    144.714793
121958     94.597429
Length: 850000, dtype: float64

In [10]:
numeric_diff_cols = [col for col in diff_cols.columns if col.endswith("_diff")]
diff_cols['sum_numeric_diff'] = diff_cols[numeric_diff_cols].sum(axis = 1)
diff_cols['mean_numeric_diff'] = diff_cols[numeric_diff_cols].mean(axis = 1)
diff_cols

,salary,salary_preds,yearsExperience_diff,sum_numeric_diff,mean_numeric_diff
891719,128,111.184496,18.083717,18.083717,18.083717
177062,116,135.742198,-22.003893,-22.003893,-22.003893
27080,133,119.792236,-2.123951,-2.123951,-2.123951
546274,126,139.158385,9.969655,9.969655,9.969655
832069,137,110.140891,23.962186,23.962186,23.962186
...,...,...,...,...,...
259178,113,118.672691,-22.003893,-22.003893,-22.003893
365838,109,108.583495,-19.902739,-19.902739,-19.902739
131932,123,88.890644,-8.204512,-8.204512,-8.204512
671155,165,150.682837,-5.968044,-5.968044,-5.968044


In [11]:
diff_cols['using_sum'] = diff_cols[['salary_preds', 'sum_numeric_diff']].sum(axis = 1)
diff_cols['using_mean'] = diff_cols[['salary_preds', 'mean_numeric_diff']].sum(axis = 1)
diff_cols

,salary,salary_preds,yearsExperience_diff,sum_numeric_diff,mean_numeric_diff,using_sum,using_mean
891719,128,111.184496,18.083717,18.083717,18.083717,129.268213,129.268213
177062,116,135.742198,-22.003893,-22.003893,-22.003893,113.738305,113.738305
27080,133,119.792236,-2.123951,-2.123951,-2.123951,117.668285,117.668285
546274,126,139.158385,9.969655,9.969655,9.969655,149.128040,149.128040
832069,137,110.140891,23.962186,23.962186,23.962186,134.103077,134.103077
...,...,...,...,...,...,...,...
259178,113,118.672691,-22.003893,-22.003893,-22.003893,96.668799,96.668799
365838,109,108.583495,-19.902739,-19.902739,-19.902739,88.680756,88.680756
131932,123,88.890644,-8.204512,-8.204512,-8.204512,80.686132,80.686132
671155,165,150.682837,-5.968044,-5.968044,-5.968044,144.714793,144.714793


In [23]:
diff_cols.salary_preds += diff_cols.sum_numeric_diff

In [24]:
diff_cols

,salary,salary_preds,yearsExperience_diff,sum_numeric_diff,mean_numeric_diff,using_sum,using_mean
891719,128,129.268213,18.083717,18.083717,18.083717,129.268213,129.268213
177062,116,113.738305,-22.003893,-22.003893,-22.003893,113.738305,113.738305
27080,133,117.668285,-2.123951,-2.123951,-2.123951,117.668285,117.668285
546274,126,149.128040,9.969655,9.969655,9.969655,149.128040,149.128040
832069,137,134.103077,23.962186,23.962186,23.962186,134.103077,134.103077
...,...,...,...,...,...,...,...
259178,113,96.668799,-22.003893,-22.003893,-22.003893,96.668799,96.668799
365838,109,88.680756,-19.902739,-19.902739,-19.902739,88.680756,88.680756
131932,123,80.686132,-8.204512,-8.204512,-8.204512,80.686132,80.686132
671155,165,144.714793,-5.968044,-5.968044,-5.968044,144.714793,144.714793


does the sum function impact the predictions too much when both the numeric columns are high magnitude in the same direction?  
(i.e. both -20 would mean the combined numeric predictors impact the salary by negative 40k dollars, which may be too much)



In [25]:
from sklearn.metrics import mean_squared_error

In [26]:
print(f"MSE using sum to combine numeric columns: {mean_squared_error(diff_cols.salary, diff_cols.using_sum)}")
print(f"MSE using mean to combine numeric columns: {mean_squared_error(diff_cols.salary, diff_cols.using_mean)}")

MSE using sum to combine numeric columns: 622.7397899817688
MSE using mean to combine numeric columns: 622.7397899817688


In [44]:
columns_to_drop = [i for i in diff_cols.columns if i.endswith(("_diff", "mean", "sum"))]

diff_cols.drop(columns = columns_to_drop)

,salary,salary_preds
891719,128,121.903485
177062,116,127.799654
27080,133,117.750164
546274,126,142.539266
832069,137,143.848817
...,...,...
259178,113,103.083661
365838,109,93.300437
131932,123,93.103601
671155,165,153.184496


In [46]:
BL.fitted_numeric_salaries.keys()

dict_keys(['yearsExperience', 'milesFromMetropolis'])

#### Pseudo code for running all combos of baseline models

- get all combinations
    - make a list of each categorical variable as an element  
    - extend the list with all 2&3 combos as well as the list of all four variables -- each element of this list is a 'category_var' argument to try  
    - for each element in the combo list  
        - for each element of the numeric_combos
            - init baseline model with the params  
            - run evaluate which spits out main metrics  
            - assemble these metrics into a list of  



In [38]:
numeric_combos = [None, 'yearsExperience', 'milesFromMetropolis', ['yearsExperience', 'milesFromMetropolis']]
category_vars = ['jobType', 'degree', 'major', 'industry']


category_combos = []
for i in range(1,5):
    combos = combinations(category_vars, i)
    for j in combos:
        category_combos.append(list(j))

category_combos

[['jobType'],
 ['degree'],
 ['major'],
 ['industry'],
 ['jobType', 'degree'],
 ['jobType', 'major'],
 ['jobType', 'industry'],
 ['degree', 'major'],
 ['degree', 'industry'],
 ['major', 'industry'],
 ['jobType', 'degree', 'major'],
 ['jobType', 'degree', 'industry'],
 ['jobType', 'major', 'industry'],
 ['degree', 'major', 'industry'],
 ['jobType', 'degree', 'major', 'industry']]

#### data output of the loop

for each category combo, turn it into the index  
columns measuring MSE for: just categories, years, miles, both, both with mean, both with sum 